In [2]:
import os
import json
import argparse
import numpy as np
import pytrec_eva
l
import glob

ModuleNotFoundError: No module named 'pytrec_eva'

In [7]:
def compute_precision_k(prediction, qrels):
    r_1_p = r_5_p = r_10_p = r_50_p = r_100_p = total = 0
    for item in prediction.keys():
        assert list(qrels[item].keys())[0] != ''

        hypo = [k for k, v in sorted(prediction[item].items(), key=lambda item: item[1], reverse=True)]
        ref = set(list(qrels[item].keys()))

        total += 1
        
        r_1_p += len(set(hypo[:1]) & ref) / len(ref)
        r_5_p += len(set(hypo[:5]) & ref) / len(ref)
        r_10_p += len(set(hypo[:10]) & ref) / len(ref)
        r_50_p += len(set(hypo[:50]) & ref) / len(ref)
        r_100_p += len(set(hypo[:100]) & ref) / len(ref)

    r_1_p = 100.0 * r_1_p / total
    r_5_p = 100.0 * r_5_p / total
    r_10_p = 100.0 * r_10_p / total
    r_50_p = 100.0 * r_50_p / total
    r_100_p = 100.0 * r_100_p / total

    print(f"Pid_Prec@1: {r_1_p: .2f}")
    # print(f"Pid_Prec@5: {r_5_p: .2f}")
    # print(f"Pid_Prec@10: {r_10_p: .2f}")
    print(f"Pid_Prec@50: {r_50_p: .2f}")
    # print(f"Pid_Prec@100: {r_100_p: .2f}")


def offical_eval(all_result, all_qrels):
    pids = list(all_result.keys())

    sqrels = dict(filter(lambda x: x[0] in pids, all_qrels.items()))
    sqrels = dict(filter(lambda x: x[1] != {"": 1}, sqrels.items()))
    sresults = dict(filter(lambda x: x[0] in pids, all_result.items()))
    # print(len(sqrels), len(sresults))
    evaluator = pytrec_eval.RelevanceEvaluator(
                    sqrels, {"recip_rank", "recall", "map", "ndcg", "ndcg_cut_3"})
    metrics = evaluator.evaluate(sresults)
    mrr_list = [v["recip_rank"] for v in metrics.values()]
    recall_5_list = [v["recall_5"] for v in metrics.values()]
    recall_10_list = [v["recall_10"] for v in metrics.values()]
    recall_20_list = [v["recall_20"] for v in metrics.values()]
    recall_30_list = [v["recall_30"] for v in metrics.values()]
    map_list = [v["map"] for v in metrics.values()]
    ndcg_list = [v["ndcg"] for v in metrics.values()]
    ndcg_3_list = [v["ndcg_cut_3"] for v in metrics.values()]


    eval_metrics = {
        "MRR": round(100*np.average(mrr_list), 2),
        "map": round(100*np.average(map_list), 2),
        "ndcg@3": round(100*np.average(ndcg_3_list), 2),
        "Recall@5": round(100*np.average(recall_5_list), 2),
        "Recall@10": round(100*np.average(recall_10_list), 2),
    }
    print(eval_metrics)
    
def compute_mrr_k(prediction, qrels):
    mrr_depths = [1,5,10]
    mrr_sums = {depth: 0.0 for depth in mrr_depths}
    for item in prediction.keys():
        assert list(qrels[item].keys())[0] != ''
        gold_pids = list(qrels[item].keys())
        hypo = [k for k, v in sorted(prediction[item].items(), key=lambda item: item[1], reverse=True)]
        positive_ranks = [i for i, pid in enumerate(hypo) if pid in gold_pids]
        if len(positive_ranks) == 0: # no gt pid in results / original retriever result
            continue
        first_positive = positive_ranks[0]
        for depth in mrr_depths:
            mrr_sums[depth] += (1.0 / (first_positive+1.0)) if first_positive < depth else 0.0
    for depth in mrr_depths:
        mrr_sums[depth] = mrr_sums[depth] / len(prediction) * 100
    print(mrr_sums)
        



In [ ]:
import json
import glob

qrels = json.load(open("{path}/qrels_test.txt"))

# since our work do not deal with first turns, we can just use the provided rewrite by the project (Enhancing Conversational Search: Large Language Model-Aided Informative Query Rewriting) and get corresponding retrieval results. The first-turn results are the same for all different approaches.
firstturn_rewrite = json.load(open('{path_to_firstturn_rewrite_result}'))

len(qrels), len(firstturn)

(16451, 1357)

In [9]:

file_path = f'./result/'

for file in sorted(glob.glob(f"{file_path}/*.json"), reverse=True):
    prediction = json.load(open(f'{file}'))
    print(file.split('/')[-1], len(prediction.keys()), len(qrels))
    # print(f'******non-first-turns: {len(prediction)}******')
    # compute_precision_k(prediction, qrels)
    # offical_eval(prediction,qrels)


    all_prediction_rewrite = prediction.copy()
    all_prediction_rewrite.update(firstturn_rewrite)


    print(f' *** rewrite {len(all_prediction_rewrite)} ***')
    compute_precision_k(all_prediction_rewrite, qrels)
    offical_eval(all_prediction_rewrite,qrels)


test_all_gpt_sft_mistral_bm25.json 6852 16451
 *** rewrite 8209 ***
Pid_Prec@1:  32.39
Pid_Prec@50:  83.67
{'MRR': 45.85, 'map': 44.39, 'ndcg@3': 43.15, 'Recall@5': 56.0, 'Recall@10': 64.92}
test_all_dpo_with_qa_loss_bm25_temp1.top50_bm25.json 6852 16451
 *** rewrite 8209 ***
Pid_Prec@1:  39.75
Pid_Prec@50:  84.99
{'MRR': 52.33, 'map': 50.76, 'ndcg@3': 49.9, 'Recall@5': 61.32, 'Recall@10': 69.06}
test_all_dpo_with_gold_reward.top50_bm25.json 6852 16451
 *** rewrite 8209 ***
Pid_Prec@1:  38.82
Pid_Prec@50:  86.76
{'MRR': 51.89, 'map': 50.33, 'ndcg@3': 49.4, 'Recall@5': 61.55, 'Recall@10': 69.79}
